# 🔍 Fraud Detection Model with Snowpark ML

## InsuranceCo - Snowflake Horizon Demo

This notebook demonstrates how Data Scientists can build ML models directly on governed data in Snowflake using Snowpark. Key points:

- **Data stays in Snowflake** - no data movement required
- **Governance policies apply** - PII is accessible only to DATA_SCIENTIST role
- **Full lineage tracking** - model inputs are traced back to source
- **Scalable compute** - leverage Snowflake warehouses for training

---

## 1. Setup and Connection

In [ ]:
# Import required libraries
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, when, lit, avg, sum as sum_, count, corr
from snowflake.snowpark.types import FloatType, IntegerType, StringType
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np

print("✅ Libraries imported successfully")

In [ ]:
# Create Snowpark session
# When running in Snowsight Notebooks, session is automatically available via get_active_session()

# For Snowsight notebooks (recommended):
session = get_active_session()

# For local development, uncomment and configure:
# connection_parameters = {
#     "account": "<your-account>",
#     "user": "<your-username>",
#     "password": "<your-password>",  # Use key-pair auth in production
#     "role": "DATA_SCIENTIST",
#     "warehouse": "INSURANCECO_ML_WH",
#     "database": "INSURANCECO",
#     "schema": "DATA_SCIENCE"
# }
# session = Session.builder.configs(connection_parameters).create()

print(f"✅ Connected to Snowflake")
print(f"   Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database: {session.get_current_database()}")

## 2. Load Governed Data

We load data from the curated `DIM_CLAIMS` table. As a DATA_SCIENTIST, we have full access to PII fields (required for fraud pattern analysis). Other roles would see masked data.

In [ ]:
# Load claims data from curated layer
claims_df = session.table("INSURANCECO.CURATED.DIM_CLAIMS")

# Load policy data for enrichment
policies_df = session.table("INSURANCECO.CURATED.DIM_POLICIES")

print(f"📊 Loaded {claims_df.count()} claims records")
print(f"📊 Loaded {policies_df.count()} policy records")

# Preview the data (PII visible because we're DATA_SCIENTIST)
print("\n🔓 DATA_SCIENTIST role can see full PII:")
claims_df.select(
    "CLAIM_ID", 
    "POLICY_HOLDER_NAME",  # PII - visible to DATA_SCIENTIST
    "POLICY_HOLDER_EMAIL", # PII - visible to DATA_SCIENTIST  
    "CLAIM_AMOUNT",
    "FRAUD_FLAG"
).show(5)

## 3. Exploratory Data Analysis

In [ ]:
# Analyze fraud distribution
fraud_dist = claims_df.group_by("FRAUD_FLAG").agg(
    count("*").alias("COUNT"),
    avg("CLAIM_AMOUNT").alias("AVG_CLAIM_AMOUNT"),
    avg("COVERAGE_UTILIZATION_PCT").alias("AVG_COVERAGE_UTIL")
)

print("📊 Fraud vs Non-Fraud Distribution:")
fraud_dist.show()

# Analyze claims that exceed coverage (potential fraud indicator)
exceeds_coverage = claims_df.filter(col("EXCEEDS_COVERAGE") == True)
print(f"\n⚠️ Claims exceeding coverage: {exceeds_coverage.count()}")
print("\n📋 Details of claims exceeding coverage:")
exceeds_coverage.select(
    "CLAIM_ID",
    "CLAIM_AMOUNT",
    "POLICY_COVERAGE_LIMIT",
    "COVERAGE_UTILIZATION_PCT",
    "FRAUD_FLAG",
    "ADJUSTER_NOTES"
).show()

## 4. Feature Engineering

Create features for the fraud detection model using Snowpark transformations.

In [ ]:
# Join claims with policies for enriched features
enriched_df = claims_df.join(
    policies_df.select(
        "POLICY_ID",
        col("RISK_SCORE").alias("POLICY_RISK_SCORE"),
        col("PREVIOUS_CLAIMS_COUNT").alias("PREV_CLAIMS"),
        col("DRIVER_AGE").alias("DRIVER_AGE"),
        col("YEARS_LICENSED").alias("YEARS_LICENSED"),
        col("POLICY_TYPE").alias("POLICY_TYPE"),
        col("PREMIUM_ANNUAL").alias("PREMIUM")
    ),
    on="POLICY_ID"
)

# Create derived features
feature_df = enriched_df.select(
    col("CLAIM_ID"),
    col("FRAUD_FLAG").cast(IntegerType()).alias("IS_FRAUD"),
    col("CLAIM_AMOUNT"),
    col("COVERAGE_UTILIZATION_PCT"),
    col("DAYS_TO_REPORT"),
    col("CLAIM_TYPE"),
    col("EXCEEDS_COVERAGE").cast(IntegerType()).alias("EXCEEDS_COVERAGE"),
    col("HIGH_VALUE_CLAIM").cast(IntegerType()).alias("HIGH_VALUE"),
    col("VEHICLE_AGE"),
    col("POLICY_RISK_SCORE"),
    col("PREV_CLAIMS"),
    col("DRIVER_AGE"),
    col("YEARS_LICENSED"),
    col("POLICY_TYPE"),
    (col("CLAIM_AMOUNT") / col("PREMIUM")).alias("CLAIM_PREMIUM_RATIO")
)

print(f"✅ Created feature DataFrame with {len(feature_df.columns)} columns")
feature_df.show(5)

## 5. Prepare Training Data

In [ ]:
# Define feature columns and target
numeric_features = [
    "CLAIM_AMOUNT", "COVERAGE_UTILIZATION_PCT", "DAYS_TO_REPORT",
    "EXCEEDS_COVERAGE", "HIGH_VALUE", "VEHICLE_AGE",
    "PREV_CLAIMS", "DRIVER_AGE", "YEARS_LICENSED", "CLAIM_PREMIUM_RATIO"
]
categorical_features = ["CLAIM_TYPE", "POLICY_RISK_SCORE", "POLICY_TYPE"]
target = "IS_FRAUD"

print(f"📊 Numeric features: {len(numeric_features)}")
print(f"📊 Categorical features: {len(categorical_features)}")
print(f"🎯 Target: {target}")

# Encode categorical variables using Snowflake ML
encoder = OneHotEncoder(
    input_cols=categorical_features,
    output_cols=[f"{c}_ENCODED" for c in categorical_features],
    drop_input_cols=True
)
encoded_df = encoder.fit(feature_df).transform(feature_df)

# Scale numeric features
scaler = StandardScaler(
    input_cols=numeric_features,
    output_cols=[f"{c}_SCALED" for c in numeric_features]
)
scaled_df = scaler.fit(encoded_df).transform(encoded_df)

# Split data into train and test sets
train_df, test_df = scaled_df.random_split([0.8, 0.2], seed=42)
print(f"\n📊 Training set: {train_df.count()} records")
print(f"📊 Test set: {test_df.count()} records")

## 6. Train Fraud Detection Model

Using Snowflake ML's RandomForestClassifier - training happens entirely within Snowflake.

In [ ]:
# Get all feature columns (scaled numeric + encoded categorical)
feature_cols = (
    [f"{c}_SCALED" for c in numeric_features] + 
    [c for c in scaled_df.columns if "_ENCODED" in c and c != target]
)

print(f"🔢 Total features for model: {len(feature_cols)}")

# Initialize and train Random Forest model
rf_model = RandomForestClassifier(
    input_cols=feature_cols,
    label_cols=[target],
    output_cols=["PREDICTION"],
    n_estimators=100,
    max_depth=10,
    random_state=42
)

print("🚀 Training Random Forest model...")
print("   (All computation happens in Snowflake - data never leaves!)")

# Train the model
rf_model.fit(train_df)
print("✅ Model training complete!")